# LTCH Pricer Program <font color = red> - CARES Act Exception </font>
----------------
## <font color = blue>Preparation Steps</font> 

**<font color = blue>Preparation Step 1-</font>** The first step of the program imports the necessary Python packages. It also creates a custom function that is used throughout the program to round values in a way that is more consistent with SAS and Excel. In addition two empty dictionaries are created for storing calculated variables and calculation summary statements.

In [541]:
import pandas as pd
import datetime
import math

def sasround(n, decimals=0):
    multiplier = 10 ** decimals
    step1=math.floor(abs(n)*multiplier + 0.5) / multiplier
    return math.copysign(step1,n)

case_number=int(input('Select a valid case number: '))
calculations={}
summary={}

Select a valid case number: 5


**<font color = blue>Preparation Step 2-</font>** Underlying rate values and factors established during the FY2020 Final Rule are read into the program and stored in a Python dictionary.

In [542]:
rates=pd.read_excel('Inputs.xlsx', sheet_name='Rates', index_col='Variable', squeeze=True).to_dict()
rates

{'Full Standard Rate': 42677.64,
 'Reduced Standard Rate': 41844.9,
 'Labor Share': 66.3,
 'Non Labor Share': 33.7,
 'Standard Fixed Loss': 26778.0,
 'SN Fixed Loss': 26552.0,
 'SN HCO BN factor': 0.949,
 'SN IPPS Adjustment': 0.954,
 'IPPS Operating Labor Rate High': 3959.1,
 'IPPS Operating Non-Labor Rate High': 1837.53,
 'IPPS Operating Labor Rate Low': 3593.91,
 'IPPS Operating Non-Labor Rate Low': 2202.7200000000003,
 'IPPS Capital Rate': 462.33,
 'IPPS UCP Reduction Factor': 0.7536}

**<font color = blue>Preparation Step 3-</font>** The FY2020 LTCH wage index for each CBSA is read into the program and stored in a Pandas dataframe. See sample.

In [543]:
ltc_wage_index_df=pd.read_excel('Inputs.xlsx', sheet_name='LTC_Wage_Index',dtype={'CBSA_Code':object})
ltc_wage_index_df.set_index('CBSA_Code', inplace=True)
ltc_wage_index_df.sample(2)

,CBSA_Title,Wage_Index
CBSA_Code,,
30780,"Little Rock-North Little Rock-Conway, AR",0.8020
25420,"Harrisburg-Carlisle, PA",0.9572


**<font color = blue>Preparation Step 4-</font>** The FY2020 IPPS wage index and GAF for each CBSA is read into the program and stored in a Pandas dataframe. See sample.

In [544]:
ipps_wage_index_df=pd.read_excel('Inputs.xlsx', sheet_name='IPPS_Wage_Index', dtype={'CBSA':object})
ipps_wage_index_df.set_index('CBSA+ST', inplace=True)
ipps_wage_index_df.sample()

,CBSA,Area_Name,ST,WI,GAF
CBSA+ST,,,,,
38940FL,38940,"Port St. Lucie, FL",FL,0.9087,0.9365


**<font color = blue>Preparation Step 5-</font>** The FY2020 LTCH and IPPS DRG weights and average length of stays are read into the program and stored in a Pandas dataframe. See randomly selected DRG.

In [545]:
drg_df=pd.read_excel('Inputs.xlsx', sheet_name='DRGs', index_col='MS-LTC-DRG')
drg_df.sample().squeeze()

MS-LTC-DRG_Title              Biopsies of musculoskeletal system & connectiv...
FY2018_LTCH_Cases                                                             1
LTC_Relative_Weight                                                      1.5478
LTC_Geometric_Average_LOS                                                  33.6
LTC_SSO_Threshold                                                            28
IPPS_Relative_Weight                                                     2.3171
IPPS_Geometric_Average_LOS                                                  5.3
Name: 478, dtype: object

**<font color = blue>Preparation Step 6-</font>** The inputs related to a particular case are read into the program and stored in a Python dictionary.

In [546]:
input_df=pd.read_excel('Inputs.xlsx', sheet_name='Case Inputs', dtype={'cbsa':object}, index_col='Case')
inputs=input_df.iloc[case_number-1].to_dict()
inputs

{'dpp_adjustment': False,
 'blend': False,
 'psch_rehab': False,
 'ipps_claim': True,
 'icu': False,
 'ventilator': True,
 'severe_wound': False,
 'spinal_cord': False,
 'state': 'AL',
 'cbsa': '13820',
 'cola': 1,
 'discharge_date': Timestamp('2020-05-14 00:00:00'),
 'LOS': 17,
 'charges': 95000.0,
 'ccr': 0.551,
 'cost': 52345.00000000001,
 'DRG': 70,
 'quality_indicator': 1,
 'ssi_pct': 0.0466,
 'mcd_pct': 0.1074,
 'ipps_residents_to_beds_ratio': 0.0,
 'ipps_residents_to_adc_ratio': 0.0,
 'beds': 89}

____________
## <font color = red>Processing Steps</font>
**<font color = red>Processing Step 1-</font>** Logic is used to determine whether a case should be paid the *standard rate* or *site neutral rate*. Due to the **Covid-19 emergency**, all discharges from LTCH hospitals whose admissions date was on or after March 1, 2020 will be paid the standard rate. If the discharge took place prior to this date, only cases that meet the following conditions will be paid the standard rate:
1. Benficiary had a severe wound or spinal cord injury 
2. Beneficiary was previously discharged from an IPPS hospital and either
    1. spent 3 days in the ICU at the IPPS hospital or
    2. spent 96 hours on a ventilator at the LTC hospital

In [547]:
calculations['admissions_date']=inputs['discharge_date']-datetime.timedelta(days=int(inputs['LOS']))
if inputs['discharge_date']<datetime.datetime(2019, 10, 1) or inputs['discharge_date']>datetime.datetime(2020, 9, 30):
    calculations['valid_date']=False
    calculations['CARES Act Flag']=False
else:
    calculations['valid_date']=True
    if calculations['admissions_date']>=datetime.datetime(2020, 3, 1):
        calculations['CARES Act Flag']=True
    else:
        calculations['CARES Act Flag']=False

if inputs['dpp_adjustment']==True:
    calculations['payment_code']='DPP'
    calculations['payment_type']='DPP'
elif calculations['CARES Act Flag']:
    calculations['payment_code']='CARES Act'
    calculations['payment_type']='Standard'
elif (inputs['severe_wound'] or inputs['spinal_cord']):    
    calculations['payment_code']='Spinal/Wound'
    calculations['payment_type']='Standard'
elif inputs['ipps_claim']:
    if (inputs['icu'] or inputs['ventilator']):
        calculations['payment_code']='IPPS discharge with icu/ventilator'
        calculations['payment_type']='Standard'
    else:
        calculations['payment_code']='IPPS discharge without icu/ventilator'
        calculations['payment_type']='Site Neutral'
else:
    calculations['payment_code']='No IPPS discharge'
    calculations['payment_type']='Site Neutral'

{key: calculations[key] for key in ['admissions_date', 'valid_date', 'CARES Act Flag', 'payment_type', 'payment_code']}

{'admissions_date': Timestamp('2020-04-27 00:00:00'),
 'valid_date': True,
 'CARES Act Flag': True,
 'payment_type': 'Standard',
 'payment_code': 'CARES Act'}

**<font color = red>Processing Step 2-</font>** Using cbsa code, logic is used to determine whether hospital is located in an urban or rural area. The cbsa code is also used to extract from the wage index dataframes the wage index and GAF factors for this particular case.

In [548]:
if len(inputs['cbsa'])==5:
    calculations['urban/rural']= 'urban'
else:
    calculations['urban/rural']= 'rural'

calculations['LTC Wage Index']=ltc_wage_index_df.loc[inputs['cbsa'],'Wage_Index']
calculations['IPPS Wage Index']=ipps_wage_index_df.loc[inputs['cbsa']+inputs['state'],'WI']
calculations['IPPS GAF']=ipps_wage_index_df.loc[inputs['cbsa']+inputs['state'],'GAF']

{key: calculations[key] for key in ['urban/rural', 'LTC Wage Index', 'IPPS Wage Index', 'IPPS GAF']}

{'urban/rural': 'urban',
 'LTC Wage Index': 0.7832,
 'IPPS Wage Index': 0.7796,
 'IPPS GAF': 0.8432}

**<font color = red>Processing Step 3-</font>** The DRG code is used to extract from the DRG dataframe the DRG title, weights, and average length of stay values.

In [549]:
calculations['DRG Title']=drg_df.loc[inputs['DRG'],'MS-LTC-DRG_Title']
calculations['DRG LTC Weight']=drg_df.loc[inputs['DRG'],'LTC_Relative_Weight']
calculations['DRG Case Count']=drg_df.loc[inputs['DRG'],'FY2018_LTCH_Cases']
calculations['LTC Geometric ALS']=drg_df.loc[inputs['DRG'],'LTC_Geometric_Average_LOS']
calculations['SSO Day Threshold']=drg_df.loc[inputs['DRG'],'LTC_SSO_Threshold']
calculations['DRG IPPS Weight']=drg_df.loc[inputs['DRG'],'IPPS_Relative_Weight']
calculations['IPPS Geometric ALS']=drg_df.loc[inputs['DRG'],'IPPS_Geometric_Average_LOS']

{key: calculations[key] for key in ['DRG Title', 'DRG LTC Weight', 'DRG Case Count', 'LTC Geometric ALS',
                                         'SSO Day Threshold', 'DRG IPPS Weight','IPPS Geometric ALS']}

{'DRG Title': 'Nonspecific cerebrovascular disorders w MCC',
 'DRG LTC Weight': 0.8629,
 'DRG Case Count': 162,
 'LTC Geometric ALS': 23.0,
 'SSO Day Threshold': 19.2,
 'DRG IPPS Weight': 1.6729,
 'IPPS Geometric ALS': 4.5}

--------------
# Calculations
--------------
## <font color = green>LTCH Standard Rate Calculations Steps</font> 
<br>
<br>
* A hospital that meets quality requirements will receive the full standard rate, while a hospital that does not meet quality requirements will receive a reduced standard rate. The rate a hospital receives is divided into a labor share and non labor share.

(1) **Labor Share of Standard Rate** = Standard Rate * Labor Share

(2) **Non Labor Share of Standard Rate** = Standard Rate * (1-Labor Share)

In [550]:
if inputs['quality_indicator']==True:
    calculations['Labor Share of Standard Rate']=sasround(rates['Full Standard Rate']*rates['Labor Share']/100,2)
    calculations['Non Labor Share of Standard Rate']=sasround(rates['Full Standard Rate']*rates['Non Labor Share']/100,2)
else:
    calculations['Labor Share of Standard Rate']=sasround(rates['Reduced Standard Rate']*rates['Labor Share']/100,2)
    calculations['Non Labor Share of Standard Rate']=sasround(rates['Reduced Standard Rate']*rates['Non Labor Share']/100,2)

{key: calculations[key] for key in ['Labor Share of Standard Rate', 'Non Labor Share of Standard Rate']}

{'Labor Share of Standard Rate': 28295.28,
 'Non Labor Share of Standard Rate': 14382.36}

<br>
* The labor share of the standard rate is adjusted by a wage index based on the hospitals location. Similarly the non labor share of the standard rate is adjusted by a cost of living adjustment (COLA) factor. Note the COLA factor is equal to 1 for all hospitals outside of Hawaii and Alaska. The adjusted labor share portion of the standard rate is added to the adjusted non labor share portion of the standard rate to arrive at a 'Geography Adjusted' standard rate value.

(3) **Wage Adjusted Labor Share of Standard Rate** = Labor Share of Standard Rate@1 * Wage Index

(4) **COLA Adjusted Non Labor Share of Standard Rate** = Non Labor Share of Standard Rate@2 * COLA Factor

(5) **Geography Adjusted Standard Rate** = Wage Adjusted Labor Share of Standard Rate@3 + COLA Adjusted Non Labor Share of Standard Rate@4


In [551]:
calculations['Wage Adjusted Labor Share of Standard Rate']=sasround(calculations['LTC Wage Index']
                                                                    *calculations['Labor Share of Standard Rate'],2)

calculations['COLA Adjusted Non Labor Share of Standard Rate']=sasround(inputs['cola']*
                                                                        calculations['Non Labor Share of Standard Rate'],2)

calculations['Geography Adjusted Standard Rate']=sasround(calculations['COLA Adjusted Non Labor Share of Standard Rate']+
                                                  calculations['Wage Adjusted Labor Share of Standard Rate'],2)

{key: calculations[key] for key in ['Wage Adjusted Labor Share of Standard Rate', 
                                    'COLA Adjusted Non Labor Share of Standard Rate', 'Geography Adjusted Standard Rate']}

{'Wage Adjusted Labor Share of Standard Rate': 22160.86,
 'COLA Adjusted Non Labor Share of Standard Rate': 14382.36,
 'Geography Adjusted Standard Rate': 36543.22}

<br>
* The final step in calculating the standard payment is to multiply by the geography adjusted standard rate by the weight associated with the MS-DRG assigned to the case.

(6) **DRG Adjusted Standard Rate** = Geography Adjusted Standard Rate@5 * DRG Weight

In [552]:
calculations['DRG Adjusted Standard Rate']=sasround(calculations['Geography Adjusted Standard Rate']*
                                                  calculations['DRG LTC Weight'],2)
{key: calculations[key] for key in ['DRG Adjusted Standard Rate']}

{'DRG Adjusted Standard Rate': 31533.14}

-----------------------------
## <font color = purple>IPPS Comparable Rate Calculations Steps</font> 

An IPPS comparable rate will be used in the payment calculation if either of the following are true. This program will calculate the IPPS comparable rate for all cases. Later on in the program, logic will be used to decide whether this value is used in the final calculation.

1. This case qualifies for the LTCH Standard rate but is a short stay outlier
2. This case does not qualify for the LTCH Standard rate and will instead receive the site neutral payment.

### <font color = purple>DSH Calculations </font>

* IPPS hospitals that qualifies as a Medicare Disproportionate Share Hospital (DSH) will receive an add on payment. Whether a hospital qualifies is dependent on its disproportiate share percentage, which is calculated by adding the percentage of Medicare inpatient days attributable to patients entitled to both Medicare Part A and Supplemental Security Income (SSI) and the percentage of total inpatient days attributable to patients eligible for Medicaid but not eligible for Medicare Part A. The two percentages that are added together are an input to the program and therefore not calculated here.

(7) **Disproportionate Share Percentage** = $\frac{Medicare SSI Days}{Total Medicare Days} + \frac{Medicaid, Nonmedicare Days}{Total Patient Days}$

In [553]:
calculations['DSH Pct']=inputs['ssi_pct']+inputs['mcd_pct']
{key: calculations[key] for key in ['DSH Pct']}

{'DSH Pct': 0.154}

<br>
* Separate disproportionate share adjustment factors are calculated for operating payments and capital payments.


* For operating payments to qualify for a DSH adjustment, the DSH percentage must be greater than or equal to 15%. If greater than or equal to 15%, the DSH percentage is adjusted by a formula dependent on whether the DSH percentage exceeds 20.2%. The result is multiplied by the uncompensated care payment (UCP) reduction factor which is set in the rules.
<br>

*If 15% <= Disproportionate Share Percentage@7 <= 20.2% then*

(8)  **DSH Operating Pct Raw Adjustment**= [.025 + [0.65 * (Disproportionate Share Percentage@7 – 0.15)]] * UCP Reduction Factor


*If Disproportionate Share Percentage@7 > 20.2% then*

(8) **DSH Operating Pct Raw Adjustment**= [.0588 + [0.825 * (Disproportionate Share Percentage@7 – 0.202)]] * UCP Reduction Factor

In [554]:
if (calculations['DSH Pct']>=.15 and calculations['DSH Pct']<=.202):
    calculations['DSH Operating Pct Raw Adjustment']=sasround((.025+(.65*(calculations['DSH Pct']-.15)))*
                                                    rates['IPPS UCP Reduction Factor'],4)
elif calculations['DSH Pct']>.202:
    calculations['DSH Operating Pct Raw Adjustment']=sasround((.0588+(.825*(calculations['DSH Pct']-.202)))*
                                                    rates['IPPS UCP Reduction Factor'],4)
else:
    calculations['DSH Operating Pct Raw Adjustment']=0
    
{key: calculations[key] for key in ['DSH Operating Pct Raw Adjustment']}

{'DSH Operating Pct Raw Adjustment': 0.0208}

<br>
* Depending on a hospital's urban/rural classification and the hospital's number of beds, the DSH Operating Adjustment factor may be capped. If both of the following conditions are met:

    1. urban with < 100 beds or rural with < 500 beds
    2. DSH Operating Pct Raw Adjustment@8 > 12% * UCP Reduction Factor

  (9) **DSH Operating Pct Capped**= .12 * UCP Adjustment Factor
  
 Otherwise
 
  (9) **DSH Operating Pct Capped**= DSH Operating Pct Raw Adjustment@8



In [555]:
if (calculations['urban/rural']=='urban' and inputs['beds']<100) or (calculations['urban/rural']=='rural' 
                                                                     and inputs['beds']<500):
    if calculations['DSH Operating Pct Raw Adjustment']>sasround(.12*rates['IPPS UCP Reduction Factor'],4):
        calculations['DSH Operating Cap Indicator']=True
        calculations['DSH Operating Pct Capped']=sasround(.12*rates['IPPS UCP Reduction Factor'],4)
    else:
        calculations['DSH Operating Cap Indicator']=False
        calculations['DSH Operating Pct Capped']=calculations['DSH Operating Pct Raw Adjustment']
else:
    calculations['DSH Operating Cap Indicator']=False
    calculations['DSH Operating Pct Capped']=calculations['DSH Operating Pct Raw Adjustment']

{key: calculations[key] for key in ['DSH Operating Cap Indicator','DSH Operating Pct Capped']}

{'DSH Operating Cap Indicator': False, 'DSH Operating Pct Capped': 0.0208}

<br>
* Only urban hospitals with greater than or equal to 100 beds qualify for a capital DSH payment. The capital adjustment factor is calculated using the formula below.

If hospital is urban with >=100 beds:

(10)**DSH Capital Pct** = exp(Disproportionate Share Percentage@7 * 0.2025)-1

else:

(10) **DSH Capital Pct**=0 

In [556]:
if (calculations['urban/rural']=='urban' and inputs['beds']>=100):
    calculations['DSH Capital Pct']=sasround(math.exp(calculations['DSH Pct']*.2025)-1,4)
else:
    calculations['DSH Capital Pct']=0

{key: calculations[key] for key in ['DSH Capital Pct']}

{'DSH Capital Pct': 0}

<br>

### <font color = purple>IME Calculations</font> 
* Hospitals that have residents in an approved graduate medical education (GME) program receive an additional payment. The indirect medical education (IME) adjustment is calculating differenly for operating payments and capital payments. The two formulas can be found below. The operating IME adjustment relies on the ratio of residents to beds which is calculated outside this program. The capital IME adjustment relies on the ratio of residents to average daily census which is also calculated outside this program.

(11) **Operating IME ADjustment** = $[(1 + \frac{residents}{beds})^.405-1]*1.35$

(12) **Capital IME ADjustment** = $exp[(\frac{residents}{average daily census})*.2822]-1$

In [557]:
calculations['Operating IME Adjustment']=sasround(((((1+inputs['ipps_residents_to_beds_ratio'])**0.405)-1)*1.35),9)
calculations['Capital IME Adjustment']=sasround(math.exp(0.2822*inputs['ipps_residents_to_adc_ratio'])-1,9)

{key: calculations[key] for key in ['Operating IME Adjustment','Capital IME Adjustment']}

{'Operating IME Adjustment': 0.0, 'Capital IME Adjustment': 0.0}

<br>

### <font color = purple>Final IPPS Calculations</font> 

* Now that the DSH and IME factors have been prepared, a final IPPS amount is ready to be calculated. The program calculates the operating amount and capital amount separately, than adds them together to calculate the full amount.


* The base rates used in the calculation of the operating amount are different depending on whether the hospitals comparable IPPS wage index is greater than 1. This is the result of the labor share being applied to the full base rate being different depding on whether the wage index is greater than 1. The operating formula is shown below:

(13) **IPPS Unadjusted Operating Amount** = [(IPPS Operating Labor Rate * IPPS Wage Index) + (IPPS Operating Non-Labor Rate * COLA)] * IPPS DRG Weight


(14) **IPPS DSH/IME Adjusted Operating Amount** = IPPS Unadjusted Operating Amount@13 * (1 + DSH Operating Pct Capped@9 + Operating IME Adjustment@11)

In [558]:
if calculations['IPPS Wage Index']>1:
    calculations['IPPS Operating Amount']=sasround(
        ((rates['IPPS Operating Labor Rate High']*calculations['IPPS Wage Index'])+
         (rates['IPPS Operating Non-Labor Rate High']*inputs['cola']))*calculations['DRG IPPS Weight']*
        (1+calculations['DSH Operating Pct Capped']+calculations['Operating IME Adjustment']),2)
else:
    calculations['IPPS Operating Amount']=sasround(
        ((rates['IPPS Operating Labor Rate Low']*calculations['IPPS Wage Index'])+
         (rates['IPPS Operating Non-Labor Rate Low']*inputs['cola']))*calculations['DRG IPPS Weight']*
        (1+calculations['DSH Operating Pct Capped']+calculations['Operating IME Adjustment']),2)

{key: calculations[key] for key in ['IPPS Operating Amount']}

{'IPPS Operating Amount': 8546.22}

<br>
* For the capital amount, an adjusted COLA factor is used in addition to the IPPS GAF. This formula compressesses the unadjusted COLA factors towards 1. The adjusted COLA factor is used along with the other factors to calculate the capital amount.

(15) **IPPS Cola Cap** = [ 0.3152* (COLA - 1)]+1

(16) **IPPS Unadjusted Capital Amount** = IPPS Capital Rate * IPPS GAF * IPPS Cola Cap@15 * IPPS DRG Weight

(17) **IPPS DSH/IME Adjusted Capital Amount** = IPPS Unadjusted Operating Amount@16 * (1 + DSH Capital Pct@10 + Capital IME Adjustment@12)

In [559]:
calculations['IPPS Cola Cap']=sasround(1+(0.3152*(inputs['cola']-1)),3)
calculations['IPPS Capital Amount']=sasround(rates['IPPS Capital Rate']*calculations['DRG IPPS Weight']*
                                             calculations['IPPS GAF']*calculations['IPPS Cola Cap']*
                                             (1+calculations['DSH Capital Pct']+calculations['Capital IME Adjustment']),2)

{key: calculations[key] for key in ['IPPS Cola Cap','IPPS Capital Amount']}

{'IPPS Cola Cap': 1.0, 'IPPS Capital Amount': 652.16}

The Final IPPS Amount is the less of the full IPPS Amount, calculated as the operating amount + the capital amount, and the IPPS per diem amount.

(18) **Full IPPS Amount** = IPPS DSH/IME Adjusted Operating Amount@14 +  IPPS DSH/IME Adjusted Capital Amount@17

(19) **PPS Per Diem Amount** = Full IPPS Amount@18 / IPPS Geometric ALS * LOS

(20) **Final IPPS Comparable Amount** = Minimum (Full IPPS Amount@18, PPS Per Diem Amount@19)

In [560]:
calculations['Full IPPS Amount']=calculations['IPPS Operating Amount']+calculations['IPPS Capital Amount']
calculations['IPPS Per Diem Amount']=sasround(calculations['Full IPPS Amount']/calculations['IPPS Geometric ALS']*
                                      inputs['LOS'],2)
calculations['Final IPPS Comparable Amount']=min(calculations['Full IPPS Amount'], calculations['IPPS Per Diem Amount'])

{key: calculations[key] for key in ['Full IPPS Amount','IPPS Per Diem Amount', 'Final IPPS Comparable Amount']}

{'Full IPPS Amount': 9198.38,
 'IPPS Per Diem Amount': 34749.44,
 'Final IPPS Comparable Amount': 9198.38}

-----------------------------
## <font color = Teal>Short Stay Outlier Calculation</font> 
This section calculates the short stay outlier payment amount. This payment amount is only applicable to cases that are being paid the standard federal rate or the blended federal rate/site neutral rate which is set to expire in FY2020. Cases that are paid strictly the neutral rate are only based on the Final IPPS Comparable amount which already accounts for short stays by taking the lesser of the full IPPS amount and the IPPS per diem amount.

Short stay outliers are paid a blended amount of the standard per diem amount (multipled by 1.2), and the Final IPPS amount. The blend is determined by the following formula (22) where SSO Day Threhold is equal to 5/6 of the geometric average length of stay for the given LTC-MS-DRG.

(21) **Standard Per Diem Amount** = $\frac{DRG Adjusted Standard Rate@6}{LTC Geometric ALS}*LOS*1.2$

(22) **LTC Blend Pct** = $\frac{LOS}{Min(25, SSO Day Threshold)}$ --> capped at 1

In [561]:
calculations['Standard Per Diem Amount']=(calculations['DRG Adjusted Standard Rate']/calculations['LTC Geometric ALS']*
                                              inputs['LOS']*1.2)

calculations['LTC Blend Pct']=sasround(min(1,inputs['LOS']/min(25,calculations['SSO Day Threshold'])),4)
calculations['IPPS Blend Pct']=1-calculations['LTC Blend Pct']

{key: calculations[key] for key in ['Standard Per Diem Amount','LTC Blend Pct', 'IPPS Blend Pct']}

{'Standard Per Diem Amount': 27968.524173913043,
 'LTC Blend Pct': 0.8854,
 'IPPS Blend Pct': 0.11460000000000004}

The full short stay outlier blended amount is equal to the following:

(23) **SSO Blend Amount** = (LTC Blend Pct@22 * Standard Per Diem Amount@21) + [(1-LTC Blend Pct@22 * Full IPPS Comparable Amount@20] 

Finally logic is used to determine whether a case had a short enough lenght of stay to qualify as a short stay outlier. As a reminder this calculation with be thrown away at the end for cases paid the full site neutral payment. If the case is a short stay outlier, than the final LTC standard base pay amount is equal to the SSO blend amount. Otherwise it is still equal to the DRG Adjusted Stanard Rate calculated in formula # 6.

(24) If LOS <=SSO Day Threshold then **LTC PPS Standard Pay Amount** = SSO Blend Amount@23

(24) If LOS > SSO Day Threshold then **LTC PPS Standard Pay Amount** = DRG Adjusted Standard Rate@6

In [562]:
calculations['LTC SSO Amount']=calculations['LTC Blend Pct']*calculations['Standard Per Diem Amount']
calculations['IPPS SSO Amount']=calculations['IPPS Blend Pct']*calculations['Final IPPS Comparable Amount']
calculations['SSO Blend Amount']=calculations['LTC SSO Amount']+calculations['IPPS SSO Amount']


if inputs['LOS']<=calculations['SSO Day Threshold']:
    calculations['SSO Flag']=True
    calculations['LTC PPS Standard Pay Amount']=calculations['SSO Blend Amount']
else:
    calculations['SSO Flag']=False
    calculations['LTC PPS Standard Pay Amount']=calculations['DRG Adjusted Standard Rate']

{key: calculations[key] for key in ['LTC SSO Amount','IPPS SSO Amount', 'SSO Blend Amount',
                                    'SSO Flag','LTC PPS Standard Pay Amount']} 

{'LTC SSO Amount': 24763.331303582607,
 'IPPS SSO Amount': 1054.1343480000003,
 'SSO Blend Amount': 25817.465651582606,
 'SSO Flag': True,
 'LTC PPS Standard Pay Amount': 25817.465651582606}

-----------------------------
## <font color = maroon>Standard High Cost Outlier Calculation</font> 
This section covers the calculation of high cost outliers for cases that are being paid the standard rate or the blended rate. The first formula calculates the cost of the case by multiplying covered charges by the hospital's cost to charter ratio (CCR).

(25) **Cost**= Charges * CCR

Next the threshold for that determines whether a case is a high cost outlier is calculated as the calculated standard pay amount and a standard fixed loss threshold determined in the rules.

(26) **LTC PPS Standard Outlier Threshold** = LTC PPS Standard Pay Amount@24 + Standard Fixed Loss Amount

If the cost exceeds this threshold, an outlier payment is calculated at 80% of costs above threshold.

(27) **Standard Outlier Payment** = (Cost@25 - LTC PPS Stanard Outlier Threshold@26) * 0.8 --> only when cost > threshold

The total standard payment amount is equal to LTC PPS Standard Pay Amount plus the outlier payment.

(28) **Total Standard Pay Amount** = LTC PPS Standard Pay Amount@24 + Standard Outlier Payment@27

In [563]:
calculations['Cost']=inputs['charges']*inputs['ccr']
calculations['LTC PPS Standard Outlier Threshold']=calculations['LTC PPS Standard Pay Amount']+rates['Standard Fixed Loss']


if calculations['Cost']>calculations['LTC PPS Standard Outlier Threshold']:
    calculations['Standard Outlier Payment Flag']=True
    calculations['Standard Outlier Payment']=(calculations['Cost']-calculations['LTC PPS Standard Outlier Threshold'])*.8
else:
    calculations['Standard Outlier Payment Flag']=False
    calculations['Standard Outlier Payment']=0

calculations['Total Standard Pay Amount']=(calculations['LTC PPS Standard Pay Amount']+
                                           calculations['Standard Outlier Payment'])

{key: calculations[key] for key in ['Cost','LTC PPS Standard Outlier Threshold', 'Standard Outlier Payment Flag',
                                    'Standard Outlier Payment','Total Standard Pay Amount']} 

{'Cost': 52345.00000000001,
 'LTC PPS Standard Outlier Threshold': 52595.46565158261,
 'Standard Outlier Payment Flag': False,
 'Standard Outlier Payment': 0,
 'Total Standard Pay Amount': 25817.465651582606}

-----------------------------
## <font color = deeppink>Site Neutral Calculation</font> 
This section covers the calculation and selection of the site neutral payment options.

The first option avaialable is the calculation of the cases cost multiplied by the site neutral high cost outlier budget neutrality factor established in the rules.

(29) **Site Neutral Cost Option** = Costs@25 * SN HCO BN Factor

The second option available is an IPPS based amount. This is calculated in multiple steps.

First the Final IPPS comparable amount is multiplied by the site neutral high cost budget neutrality factor established in the rules.

(30) **Site Neutral IPPS Base Pay** = Final IPPS Comparable Amount@20  * SN HCO BN Factor

Next a site neutral outlier threshold is established and compared against the cost to determine whether this case qualifies for a high cost outlier payment. Note that the Final IPPS Comparable amount is added to the fixed loss amount without the budget neutrality factor appplied.

(31) **Site Neutral Outlier Thresholdhold** = Final IPPS Comparable Amount@20 + Site Neutral Fixed Loss Amount

If the cost exceeds this threshold, an outlier payment is calculated at 80% of costs above threshold.

(32) **Site Neutral Outlier Payment** = (Cost@25 - Site Neutral Outlier Threshold@31) * 0.8 --> only when cost > threshold

The total Site Neutral IPPS Options is equal to to the Site Neutral IPPS Base Pay plus the Site Neutral Outlier Payment. THis amount is then multiplied by a Site Neutral IPPS Adjustment factor established in the rules.

(33) **Site Neutral IPPS Option** = (Site Neutral IPPS Base Pay@30 + Site Neutral Outlier Payment@32) * SN IPPS Adjustment Factor

The determined Site Neutral payment amount is the lesser of the two options:

(34) **Total Site Neutral Amount** = Minimum of (Site Neutral Cost Option@29, Site Neutral IPPS Option @33)

For hospitals that are still being paid a transition rate for site neutral cases, the transition amount is calculated as the average of the site neutral amount and the standard pay amount.

(35) **Total Transition Amount** = (Total Standard Pay Amount@28 + Total Site Neutral Amount@34) / 2




In [564]:
calculations['Site Neutral Cost Option']=sasround(calculations['Cost']*rates['SN HCO BN factor'],2)

calculations['Site Neutral IPPS Base Pay']=sasround(calculations['Final IPPS Comparable Amount']*
                                                    rates['SN HCO BN factor'],2)

calculations['Site Neutral Outlier Threshold']=sasround(calculations['Final IPPS Comparable Amount']+
                                                        rates['SN Fixed Loss'],2)

if calculations['Cost']>calculations['Site Neutral Outlier Threshold']:
    calculations['Site Neutral Outlier Payment Flag']=True
    calculations['Site Neutral Outlier Payment']=sasround((calculations['Cost']-
                                                           calculations['Site Neutral Outlier Threshold'])*.8,2)
else:
    calculations['Site Neutral Outlier Payment Flag']=False
    calculations['Site Neutral Outlier Payment']=0

    
calculations['Site Neutral IPPS Option']=sasround((calculations['Site Neutral IPPS Base Pay']+
                                          calculations['Site Neutral Outlier Payment'])*rates['SN IPPS Adjustment'],2)

if calculations['Site Neutral Cost Option']<=calculations['Site Neutral IPPS Option']:
    calculations['Site Neutral Selection']="Cost Option"
    calculations['Total Site Neutral Amount']=calculations['Site Neutral Cost Option']
else:
    calculations['Site Neutral Selection']="IPPS Option"
    calculations['Total Site Neutral Amount']=calculations['Site Neutral IPPS Option']

calculations['Total Transition Amount']=(calculations['Total Standard Pay Amount']+
                                         calculations['Total Site Neutral Amount'])/2

{key: calculations[key] for key in ['Site Neutral Cost Option','Site Neutral IPPS Base Pay',
                                    'Site Neutral Outlier Threshold', 'Site Neutral Outlier Payment Flag',
                                    'Site Neutral Outlier Payment','Site Neutral IPPS Option','Site Neutral Selection',
                                   'Total Site Neutral Amount', 'Total Transition Amount']} 

{'Site Neutral Cost Option': 49675.41,
 'Site Neutral IPPS Base Pay': 8729.26,
 'Site Neutral Outlier Threshold': 35750.38,
 'Site Neutral Outlier Payment Flag': True,
 'Site Neutral Outlier Payment': 13275.7,
 'Site Neutral IPPS Option': 20992.73,
 'Site Neutral Selection': 'IPPS Option',
 'Total Site Neutral Amount': 20992.73,
 'Total Transition Amount': 23405.0978257913}

-----------------------------
## <font color = slategray>DPP Calculation</font> 
This section covers the calculation of payments for hospitals under the DPP policy.

The base payment for these cases is equal to the Final IPPS amount.

(35) **DPP Base Pay**= Full IPPS Amount@18

An outlier threshold is established by adding the IPPS SN fixed loss amount to the DPP Base pay.

(36) **DPP Outlier Thresholdhold** = DPP Base Pay@20 + Site Neutral Fixed Loss Amount

If the cost exceeds this threshold, an outlier payment is calculated at 80% of costs above threshold.

(37) **DPP Outlier Payment** = (Cost@25 - DPP Outlier Threshold@31) * 0.8 --> only when cost > threshold

The total DPP Payment is equal to to the DPP Base Pay plus the DPP Outlier Payment.

(38) **DPP Total Payment** = DPP Base Pay@35 + DPP Outlier Payment@37

In [565]:
calculations['DPP Base Pay']=calculations['Full IPPS Amount']

calculations['DPP Outlier Threshold']=sasround(calculations['DPP Base Pay']+ rates['SN Fixed Loss'],2)

if calculations['Cost']>calculations['DPP Outlier Threshold']:
    calculations['DPP Outlier Payment Flag']=True
    calculations['DPP Outlier Payment']=sasround((calculations['Cost']-
                                                           calculations['DPP Outlier Threshold'])*.8,2)
else:
    calculations['DPP Outlier Payment Flag']=False
    calculations['DPP Outlier Payment']=0

    
calculations['DPP Total Payment']=sasround((calculations['DPP Base Pay'] + calculations['DPP Outlier Payment']),2)

{key: calculations[key] for key in ['DPP Base Pay','DPP Outlier Threshold','DPP Outlier Payment Flag','DPP Outlier Payment',
                                    'DPP Total Payment']}

{'DPP Base Pay': 9198.38,
 'DPP Outlier Threshold': 35750.38,
 'DPP Outlier Payment Flag': True,
 'DPP Outlier Payment': 13275.7,
 'DPP Total Payment': 22474.08}

--------------
# Final Payment Reporting

The final payment amount is assigned based on payment type. The full calculation dictionary is printed.

In [566]:
if calculations['payment_type']=='Standard':
    calculations['Final Payment']=calculations['Total Standard Pay Amount']
    
elif (calculations['payment_type']=='Site Neutral' and inputs['blend']==True):
    calculations['Final Payment']=calculations['Total Transition Amount']

elif (calculations['payment_type']=='Site Neutral' and inputs['blend']==False):
    calculations['Final Payment']=calculations['Total Site Neutral Amount']
elif calculations['payment_type']=='DPP':
    calculations['Final Payment']=calculations['DPP Total Payment']

In [571]:
calculations

{'admissions_date': Timestamp('2020-04-27 00:00:00'),
 'valid_date': True,
 'CARES Act Flag': True,
 'payment_code': 'CARES Act',
 'payment_type': 'Standard',
 'urban/rural': 'urban',
 'LTC Wage Index': 0.7832,
 'IPPS Wage Index': 0.7796,
 'IPPS GAF': 0.8432,
 'DRG Title': 'Nonspecific cerebrovascular disorders w MCC',
 'DRG LTC Weight': 0.8629,
 'DRG Case Count': 162,
 'LTC Geometric ALS': 23.0,
 'SSO Day Threshold': 19.2,
 'DRG IPPS Weight': 1.6729,
 'IPPS Geometric ALS': 4.5,
 'Labor Share of Standard Rate': 28295.28,
 'Non Labor Share of Standard Rate': 14382.36,
 'Wage Adjusted Labor Share of Standard Rate': 22160.86,
 'COLA Adjusted Non Labor Share of Standard Rate': 14382.36,
 'Geography Adjusted Standard Rate': 36543.22,
 'DRG Adjusted Standard Rate': 31533.14,
 'DSH Pct': 0.154,
 'DSH Operating Pct Raw Adjustment': 0.0208,
 'DSH Operating Cap Indicator': False,
 'DSH Operating Pct Capped': 0.0208,
 'DSH Capital Pct': 0,
 'Operating IME Adjustment': 0.0,
 'Capital IME Adjustmen